In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [3]:
rain = pd.read_csv("../Data/Rainfall.csv")

In [4]:
import glob

slp_path = "../Data/Anomaly/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['../Data/Anomaly/slp\\slp1.csv', '../Data/Anomaly/slp\\slp10.csv', '../Data/Anomaly/slp\\slp11.csv', '../Data/Anomaly/slp\\slp12.csv', '../Data/Anomaly/slp\\slp2.csv', '../Data/Anomaly/slp\\slp3.csv', '../Data/Anomaly/slp\\slp4.csv', '../Data/Anomaly/slp\\slp5.csv', '../Data/Anomaly/slp\\slp6.csv', '../Data/Anomaly/slp\\slp7.csv', '../Data/Anomaly/slp\\slp8.csv', '../Data/Anomaly/slp\\slp9.csv']


In [5]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

857.1


In [6]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
# month_data.head()
# len(month_data)

In [7]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

combine = []
for i in range(12):
  month_path = slp_months[i]
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [8]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [9]:
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(97)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [10]:
model_1 = get_model_1()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense (Dense)               (None, 324)               105300    
                                                                 
 re_lu (ReLU)                (None, 324)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                31525     
                                                                 
 dense_2 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/800
40/40 - 1s - loss: nan - val_loss: nan - 703ms/epoch - 18ms/step
Epoch 2/800
40/40 - 0s - loss: nan - val_loss: nan - 176ms/epoch - 4ms/step
Epoch 3/800
40/40 - 0s - loss: nan - val_loss: nan - 191ms/epoch - 5ms/step
Epoch 4/800
40/40 - 0s - loss: nan - val_loss: nan - 194ms/epoch - 5ms/step
Epoch 5/800
40/40 - 0s - loss: nan - val_loss: nan - 186ms/epoch - 5ms/step
Epoch 6/800
40/40 - 0s - loss: nan - val_loss: nan - 184ms/epoch - 5ms/step
Epoch 7/800
40/40 - 0s - loss: nan - val_loss: nan - 164ms/epoch - 4ms/step
Epoch 8/800
40/40 - 0s - loss: nan - val_loss: nan - 157ms/epoch - 4ms/step
Epoch 9/800
40/40 - 0s - loss: nan - val_loss: nan - 167ms/epoch - 4ms/step
Epoch 10/800
40/40 - 0s - loss: nan - val_loss: nan - 187ms/epoch - 5ms/step
Epoch 11/800
40/40 - 0s - loss: nan - val_loss: nan - 171ms/epoch - 4ms/step
Epoch 12/800
40/40 - 0s - loss: nan - val_loss: nan - 189ms/epoch - 5ms/step
Epoch 13/800
40/40 - 0s - loss: nan - val_loss: nan - 175ms/epoch - 4ms/step
Epoch 1

In [12]:
print(history)

In [13]:
model_1.layers[3].get_weights()[0].shape

(324, 97)

In [14]:
model_temp1 = Model(inputs=model_1.input, outputs=model_1.layers[3].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [15]:
# define encoder
def get_model_2(n_inputs=97):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(29)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [16]:
model_2 = get_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 97)]              0         
                                                                 
 dense_3 (Dense)             (None, 97)                9506      
                                                                 
 re_lu_1 (ReLU)              (None, 97)                0         
                                                                 
 dense_4 (Dense)             (None, 29)                2842      
                                                                 
 dense_5 (Dense)             (None, 97)                2910      
                                                                 
Total params: 15,258
Trainable params: 15,258
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

Epoch 1/800
40/40 - 0s - loss: nan - val_loss: nan - 453ms/epoch - 11ms/step
Epoch 2/800
40/40 - 0s - loss: nan - val_loss: nan - 113ms/epoch - 3ms/step
Epoch 3/800
40/40 - 0s - loss: nan - val_loss: nan - 122ms/epoch - 3ms/step
Epoch 4/800
40/40 - 0s - loss: nan - val_loss: nan - 129ms/epoch - 3ms/step
Epoch 5/800
40/40 - 0s - loss: nan - val_loss: nan - 122ms/epoch - 3ms/step
Epoch 6/800
40/40 - 0s - loss: nan - val_loss: nan - 121ms/epoch - 3ms/step
Epoch 7/800
40/40 - 0s - loss: nan - val_loss: nan - 121ms/epoch - 3ms/step
Epoch 8/800
40/40 - 0s - loss: nan - val_loss: nan - 122ms/epoch - 3ms/step
Epoch 9/800
40/40 - 0s - loss: nan - val_loss: nan - 125ms/epoch - 3ms/step
Epoch 10/800
40/40 - 0s - loss: nan - val_loss: nan - 124ms/epoch - 3ms/step
Epoch 11/800
40/40 - 0s - loss: nan - val_loss: nan - 126ms/epoch - 3ms/step
Epoch 12/800
40/40 - 0s - loss: nan - val_loss: nan - 122ms/epoch - 3ms/step
Epoch 13/800
40/40 - 0s - loss: nan - val_loss: nan - 123ms/epoch - 3ms/step
Epoch 1

In [18]:
model_temp2 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [19]:
def get_model_3(n_inputs=29):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  output = Dense(n_inputs, activation='linear')(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
  return model

In [20]:
model_3 = get_model_3()
model_3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 29)]              0         
                                                                 
 dense_6 (Dense)             (None, 29)                870       
                                                                 
 re_lu_2 (ReLU)              (None, 29)                0         
                                                                 
 dense_7 (Dense)             (None, 9)                 270       
                                                                 
 dense_8 (Dense)             (None, 29)                290       
                                                                 
Total params: 1,430
Trainable params: 1,430
Non-trainable params: 0
_________________________________________________________________


In [21]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))
# plot loss

Epoch 1/800
40/40 - 1s - loss: nan - val_loss: nan - 538ms/epoch - 13ms/step
Epoch 2/800
40/40 - 0s - loss: nan - val_loss: nan - 158ms/epoch - 4ms/step
Epoch 3/800
40/40 - 0s - loss: nan - val_loss: nan - 143ms/epoch - 4ms/step
Epoch 4/800
40/40 - 0s - loss: nan - val_loss: nan - 138ms/epoch - 3ms/step
Epoch 5/800
40/40 - 0s - loss: nan - val_loss: nan - 130ms/epoch - 3ms/step
Epoch 6/800
40/40 - 0s - loss: nan - val_loss: nan - 130ms/epoch - 3ms/step
Epoch 7/800
40/40 - 0s - loss: nan - val_loss: nan - 126ms/epoch - 3ms/step
Epoch 8/800
40/40 - 0s - loss: nan - val_loss: nan - 144ms/epoch - 4ms/step
Epoch 9/800
40/40 - 0s - loss: nan - val_loss: nan - 130ms/epoch - 3ms/step
Epoch 10/800
40/40 - 0s - loss: nan - val_loss: nan - 126ms/epoch - 3ms/step
Epoch 11/800
40/40 - 0s - loss: nan - val_loss: nan - 127ms/epoch - 3ms/step
Epoch 12/800
40/40 - 0s - loss: nan - val_loss: nan - 128ms/epoch - 3ms/step
Epoch 13/800
40/40 - 0s - loss: nan - val_loss: nan - 145ms/epoch - 4ms/step
Epoch 1

In [22]:
def get_model_final(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs)(visible)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  e = Dense(29)(e)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  e = Dense(29)(bottleneck)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear')(e)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [23]:
model_final = get_model_final()
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_9 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_3 (ReLU)              (None, 324)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [24]:
model_final.layers[2] = model_1.layers[3]
model_final.layers[11] = model_1.layers[4]
model_final.layers[4] = model_2.layers[3]
model_final.layers[9] = model_2.layers[4]
model_final.layers[6] = model_3.layers[3]
model_final.layers[7] = model_3.layers[4]
model_final.compile(optimizer='adam', loss='mse')

In [25]:
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_9 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_3 (ReLU)              (None, 324)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                31525     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [26]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))


Epoch 1/1000
40/40 - 1s - loss: nan - val_loss: nan - 761ms/epoch - 19ms/step
Epoch 2/1000
40/40 - 0s - loss: nan - val_loss: nan - 197ms/epoch - 5ms/step
Epoch 3/1000
40/40 - 0s - loss: nan - val_loss: nan - 225ms/epoch - 6ms/step
Epoch 4/1000
40/40 - 0s - loss: nan - val_loss: nan - 193ms/epoch - 5ms/step
Epoch 5/1000
40/40 - 0s - loss: nan - val_loss: nan - 208ms/epoch - 5ms/step
Epoch 6/1000
40/40 - 0s - loss: nan - val_loss: nan - 190ms/epoch - 5ms/step
Epoch 7/1000
40/40 - 0s - loss: nan - val_loss: nan - 164ms/epoch - 4ms/step
Epoch 8/1000
40/40 - 0s - loss: nan - val_loss: nan - 171ms/epoch - 4ms/step
Epoch 9/1000
40/40 - 0s - loss: nan - val_loss: nan - 188ms/epoch - 5ms/step
Epoch 10/1000
40/40 - 0s - loss: nan - val_loss: nan - 172ms/epoch - 4ms/step
Epoch 11/1000
40/40 - 0s - loss: nan - val_loss: nan - 178ms/epoch - 4ms/step
Epoch 12/1000
40/40 - 0s - loss: nan - val_loss: nan - 175ms/epoch - 4ms/step
Epoch 13/1000
40/40 - 0s - loss: nan - val_loss: nan - 182ms/epoch - 5ms

In [27]:
# model_final.save("../Data/Saved Models/model_final_slp.h5")
# model_final = keras.models.load_model('./Data/Models/model_final.h5')

In [28]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return tf.matmul(inputs, self.w) + self.b
        except: return tf.matmul(inputs, self.w)

In [29]:
std = []
for i in model_final.layers:
    # print(i.weights[0])
    try: std.append(np.array(math.reduce_std(i.weights[0])) * 1.9)
    except: std.append([])

In [30]:
final_layers = keras.Sequential([Input(shape=(None, 324))])
for i, layer in enumerate(model_final.layers):
    try:
        temp_layer = []
        for ind, j in enumerate(layer.weights[0]):
            temp_weights = []
            for index, k in enumerate(j):
                if k < std[i]: temp_weights.append(0)
                else:          temp_weights.append(1)  
            temp_layer.append(temp_weights)
        weights = tf.Variable(temp_layer, dtype='float32')
        try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
        except: final_layers.add(Custom_Layer(weights))
    except IndexError: final_layers.add(layer)

In [31]:
final_layers.compile(optimizer='adam', loss='mse')
final_layers.build(input_shape=(None, 324))
final_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        multiple                  0         
                                                                 
 custom__layer_1 (Custom_Lay  (None, None, 324)        104976    
 er)                                                             
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 custom__layer_3 (Custom_Lay  (None, None, 97)         31428     
 er)                                                             
                                                                 
 re_lu_4 (ReLU)              multiple                  0         
                                                                 
 custom__layer_5 (Custom_Lay  (None, None, 29)         2

In [32]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 324), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")


In [33]:
final_layers1 = Model(inputs=final_layers.input, outputs=final_layers.layers[3].output)
final_layers1.trainable = False

final_layers2 = Model(inputs=final_layers.input, outputs=final_layers.layers[5].output)
final_layers2.trainable = False

final_layers3 = Model(inputs=final_layers.input, outputs=final_layers.layers[7].output)
final_layers3.trainable = False

models = [final_layers1, final_layers2, final_layers3]

In [34]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  try: pred_m = models[mod](feature)
  except: 
    print(i)
    raise(IndexError)
  pred_f = pred_m.numpy()
  return pred_f

In [35]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [36]:
from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9
# models = [model_final1, model_final2, model_final3]
models = [final_layers1, final_layers2, final_layers3]

minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')

ValueError: x and y must have the same length.